In [4]:
import json
import random
random.seed(42)

with open("./tmp/v1.0-simplified-nq-dev-all.jsonl", "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f.readlines()]
    print(len(data))
    # dict: id, question
    data = [
        {
            "id": i,
            "user_query": data[i]["question_text"],
        }
        for i in range(len(data))
    ]
    with open("./data/NaturalQuestions.json", "w", encoding="utf-8") as g:
        json.dump(data, g, indent=2, ensure_ascii=False)
        
    data = random.sample(data, 100)
    data = sorted(data, key=lambda x: x["id"])
    
    with open("./NaturalQuestions.json", "w", encoding="utf-8") as g:
        json.dump(data, g, indent=2, ensure_ascii=False)

1494


In [2]:
import json

with open("./tmp/LearningQ/data/experiments/teded/tgt-test.txt", "r", encoding="utf-8") as f:
    data = [line.strip() for line in f.readlines()]
    data = [i for i in data if i.endswith("?")]
    data = [
        {
            "id": i,
            "user_query": data[i],
        }
        for i in range(len(data))
    ]
    with open("./LearningQ.json", "w", encoding="utf-8") as g:
        json.dump(data, g, indent=2, ensure_ascii=False)

In [2]:
import json

model = "vicuna-13b"
files = [
            "BioASQ",
            "nfcorpus",
            "FiQA-2018",
            "SciFact",
            "LearningQ",
            "NaturalQuestions",
            "HotpotQA",
        ]

for file in files:

    with open(f"./{model}/{file}.json", "r", encoding="utf-8") as f:
        add = json.load(f)
    with open(f"./data/{file}.json", "r", encoding="utf-8") as f:
        data = json.load(f)

    for i in range(len(data)):
        data[i][f"{model}_response"] = add[i][f"{model}_response"]

    with open(f"./data/{file}_.json", "w", encoding="utf-8") as g:
        json.dump(data, g, indent=2, ensure_ascii=False)

In [4]:
with open("./data/SciFact_add.json", "r", encoding="utf-8") as f:
    data = json.load(f)
    print(len(data))
    data = [
        {
            "id": i,
            "user_query": data[i]["user_query"],
            "chatgpt_response": data[i]["chatgpt_response"],
            "text-davinci-002_response": data[i]["text-davinci-002_response"],
            "text-davinci-003_response": data[i]["text-davinci-003_response"],
            "llama-2-7b-chat-hf_response": data[i]["llama-2-7b-chat-hf_response"],
            "llama-2-13b-chat-hf_response": data[i]["llama-2-13b-chat-hf_response"],
            "alpaca-7b_response": data[i]["alpaca-7b_response"],
        }
        for i in range(len(data))
    ]
    with open("./data/SciFact_add_.json", "w", encoding="utf-8") as g:
        json.dump(data, g, indent=2, ensure_ascii=False)

5235


In [1]:
# read judge_part files
import json
import os

for path in os.listdir("./judge_part"):
    with open(f"./judge_part/{path}", "r", encoding="utf-8") as f:
        data = json.load(f)
        data = sorted(data, key=lambda x: x["id"])
        with open(f"./judge_part/{path}_", "w", encoding="utf-8") as g:
            json.dump(data, g, indent=2, ensure_ascii=False)

In [ ]:
import os
import json

for path in os.listdir("./data"):
    with open(f"./data/{path}", "r", encoding="utf-8") as f:
        data = json.load(f)
        # for key end with "response", clear the multiple "\n" of its value, and save it.
        for i in range(len(data)):
            for key in data[i].keys():
                if key.endswith("response"):
                    ans = data[i][key]
                    ans = ans.strip().split("\n")
                    ans = "\n".join([_ for _ in ans if _])
                    data[i][key] = ans
        with open(f"./new/{path}", "w", encoding="utf-8") as g:
            json.dump(data, g, indent=2, ensure_ascii=False)

In [1]:
import json

path = "./data"
# files = [
    # "BioASQ",
    # "nfcorpus",
    # "FiQA-2018",
    # "SciFact",
    # "LearningQ",
    # "NaturalQuestions",
    # "HotpotQA",
# ]
with open(f"{path}/Open-Domain.json", "r", encoding="utf-8") as f:
    data = json.load(f)
with open(f"{path}/halueval.json", "r", encoding="utf-8") as f:
    add = json.load(f)
data.extend(add)
for i in range(len(data)):
    data[i]["id"] = i
with open(f"./data/Open-Domain.json", "w", encoding="utf-8") as g:
    json.dump(data, g, indent=2, ensure_ascii=False)

In [1]:
import json

with open("./add/Open-Domain.json", "r", encoding="utf-8") as f:
    data = json.load(f)
data = sorted(data, key=lambda x: x["id"])
with open("./add/Open-Domain.json", "w", encoding="utf-8") as g:
    json.dump(data, g, indent=2, ensure_ascii=False)

In [3]:
import json

with open("../hallu/data/hallu_add/qa.json", "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f.readlines()][:20]
    print(len(data))
    # json file
    data = [
        {
            "id": i,
            "user_query": data[i]["question"].strip(),
        }
        for i in range(len(data))
    ]
with open("./Open-Domain_demo.json", "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

20


In [9]:
import os
import json
from main import check_exist

id_path = "./pure/data/"
data_path = "./response/llama-2-7b-chat-hf_top-p/"
save_path = "./pure/llama-2-7b-chat-hf_top-p/"
for path in os.listdir(id_path):
    if path in [
        # "Bio-Medical.json",
        # "Finance.json",
        # "Science.json",
        # "Education.json",
        # "Open-Domain.json",
    ]:
        continue
    with open(os.path.join(id_path, path), "r", encoding="utf-8") as f:
        data = json.load(f)
        ids = [i["id"] for i in data]
    with open(os.path.join(data_path, path), "r", encoding="utf-8") as f:
        gpt = json.load(f)
    gpt = [i for i in gpt if i["id"] in ids]
    gpt = sorted(gpt, key=lambda x: x["id"])
    check_exist(save_path)
    with open(os.path.join(save_path, path), "w", encoding="utf-8") as g:
        json.dump(gpt, g, indent=2, ensure_ascii=False)